In [1]:
!pip install gymnasium renderlab

In [2]:
!pip install opencv-python

In [3]:
!pip install pygame

In [4]:
import gymnasium as gym
import random
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time

In [5]:
#visualise maze:

desc=["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]
mazeSize = [len(desc),len(desc[0])]

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human") 

In [35]:
# Q-LEARNING FUNCTIONS

epsilonValue = 0.05
alpha = 0.5
qTable = {}
currentState = 0

def resetTable():
    global qTable
    qTable = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        qTable[i] = [0,0,0,0] 
    global currentState
    currentState = 0

def calcPossibleMoves(state):
    global qTable
    possibleMoves = []
    if (state+1) % mazeSize[1] != 0:
        possibleMoves.append(2)
        
    if (state+1) % mazeSize[1] != 1:
        if not state == 0:
            possibleMoves.append(0)
        
    if state > (mazeSize[1]-1):
        if not state == 0:
            possibleMoves.append(3)
    
    if state < ((mazeSize[0] * mazeSize[1]) - mazeSize[1]):
        possibleMoves.append(1)
        
    for value in qTable[state]:
        if value < 0 and abs(value) > epsilonValue:
            if qTable[state].index(value) in possibleMoves:
                possibleMoves.remove(qTable[state].index(value))
        
    return possibleMoves

def nextStep(state):
    global qTable
    
    x = max(qTable[state])
    if x >= epsilonValue:
        nextMove = qTable[state].index(x)
    else:
        possMoves = calcPossibleMoves(state)
        nextMove = possMoves[random.randrange(len(possMoves))]
    return nextMove

convergenceThresh = 0.005
def updateTable(direction, nextState, reward):
    global qTable
    didConverge = False
    global currentState
    updated = qTable[currentState][direction] + alpha*(reward + max(qTable[nextState]) - qTable[currentState][direction])
    changeInQ = abs(qTable[currentState][direction] - updated)
    if changeInQ < convergenceThresh:
        if changeInQ > 0:
            didConverge = True
    qTable[currentState][direction] = updated
    currentState = nextState
    return didConverge, changeInQ

In [38]:
maxEpisodes = 1000
currentEpisode = 1
converged = False

resetTable()
env.reset()
start_time = time.time()
while currentEpisode <= maxEpisodes:
    if converged:
        break
        
    direction = nextStep(currentState)
    observation, reward, terminated, truncated, info = env.step(direction)
    
    if terminated:
        if reward < 1:
            reward = -1
    
    converged, changeInQ = updateTable(direction, observation, reward)

    if terminated or truncated or converged:
        observation, info = env.reset()
        if not converged:
            currentEpisode += 1

        
    if converged:
        end_time = time.time()

    clear_output(wait=True)
    print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
    print("Time: " + str(round(time.time()-start_time, 3)) + " sec")
    print("Q-Table:")
    for i in range(len(qTable)):
        print(str(i) + ": " + str(qTable[i]))
    print("change in Q: " + str(changeInQ))
            
if converged:
    duration = end_time - start_time
    print(str(round(duration, 3)) + " seconds to converge")
else:
    print("No convergence")

Episode: 18/1000
Time: 162.004 sec
Q-Table:
0: [0, 0.015625, 0.00390625, 0]
1: [0.0078125, -0.5, 0.0, 0]
2: [0.0, -0.5, 0.0, 0]
3: [0.0, -0.5, 0, 0]
4: [0, 0.03125, -0.5, 0.0078125]
5: [0, 0.0, 0.0, 0]
6: [0.0, 0.0, 0, 0]
7: [0, 0.0, 0, 0.0]
8: [0, -0.5, 0.1875, 0.0]
9: [0.0, 0.5, 0.0, -0.5]
10: [0.0, -0.5, 0.0, -0.5]
11: [0.0, 0.0, 0, -0.5]
12: [0, 0, 0.0, 0.0]
13: [-0.5, 0.8125, 0, 0.0]
14: [0.0, 0, 0, 0]
15: [-0.5, 0, 0, 0.0]
16: [0, 0, 0, 0]
17: [0, 0, 0.96875, 0]
18: [0.0, 0, 0.0, 0.0]
19: [0, 0, 0, 0]
change in Q: 0.00390625
162.003 seconds to converge


In [30]:
env.reset()

(0, {'prob': 1})

In [39]:
env.close()